In [1]:
import os
# Ensure the .outputs directory exists
OUTPUTS_DIR = '.outputs'
INPUTS_DIR = '.inputs'

os.makedirs(OUTPUTS_DIR, exist_ok=True)
os.makedirs(INPUTS_DIR, exist_ok=True)

def read_document(path):
    """Reads the content of a document."""
    with open(path, 'r', encoding='utf-8') as file:
        return file.read().strip()

def write_document(path, content):
    """Writes content to a document."""
    with open(path, 'w', encoding='utf-8') as file:
        file.write(content)

# Dummy implementations of the pipeline steps
def step_00_extract_schema():
    """Extracts the schema and returns categories_to_evaluate."""
    # Replace with actual implementation
    categories = read_document(os.path.join(OUTPUTS_DIR, 'categories_to_evaluate.txt'))
    return categories

In [5]:
import uuid
categories_path = os.path.join(OUTPUTS_DIR, 'categories_to_evaluate.txt')
categories_to_evaluate = read_document(categories_path)
thread_id_step1 = str(uuid.uuid4())
try:
    new_cks = read_document(os.path.join(OUTPUTS_DIR, 'new_cks.txt'))
except:
    new_cks = ''

#### COLD START.

Not personalized but a way to get the assessments for all the class in order to get the first knowledge state in the next iteration.

In [18]:
import boto3
import json

#AWS credentials
# Initialize boto3 Lambda client
lambda_client = boto3.client(
    'lambda',
    region_name='us-east-1',
    aws_access_key_id=AWS_ACCESS_KEY,  # Replace with your access key
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY  # Replace with your secret key
)

# Define the payload to send to the Lambda function
payload = {
        "lesson": read_document(os.path.join(INPUTS_DIR, 'lesson.txt')),
        "goals_and_criteria": read_document(os.path.join(INPUTS_DIR, 'goals_and_criteria.txt')),
        "user_context": "Grade 6 classroom",
        "studentid": "student1",
        "lessonid": "lesson0",
        "current_knowledge_state": new_cks if new_cks else "No student CKS is available",
        "old_cks": "Not yet provided",
        "categories_to_evaluate": categories_to_evaluate,
        "last_assessment": read_document(os.path.join(INPUTS_DIR, 'assessment_to_eval.txt')),
        "thread_id": thread_id_step1
    }

#TUTOR SYS
# Invoke the Lambda function
try:
    response_step_2 = lambda_client.invoke(
        FunctionName='tutorsys-s-analyst',  # Replace with your Lambda function name
        InvocationType='RequestResponse',  # Set to 'Event' for async
        Payload=json.dumps(payload)
    )

    # Process the response
    response_payload_step_2 = json.loads(response_step_2['Payload'].read())
    print("Response:", response_payload_step_2)
except Exception as err:
    print(f"Error invoking Lambda function: {err}")

File not found: .inputs/assessment_to_eval.txt
Response: {'statusCode': 200, 'headers': {'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*'}, 'body': "**Strategic Plan for Assessment and Learning**\n\n0. **Small description of CKS if exist**: No Current Knowledge State (CKS) is available for the student.\n\n1. **Key Learning Goal**: \n   - To develop an understanding of the various dimensions relevant to agent design and operation in artificial intelligence, focusing on modularity, learning, decision-making under uncertainty, and interaction with other agents.\n\n2. **Feedback**: \n   - *Student's Strengths*: Not applicable due to the absence of assessment data.\n   - *Student's Points of Improvements*:\n     - **Modularity**: Understanding of how agents can break down tasks into smaller components and interact.\n     - **Planning Horizon**: Grasping the importance of both short-term and long-term planning in decision-making.\n     - **Representation**: Learning how

In [24]:
strategic_doc = response_payload_step_2['body']
thread_id_step2 = read_document(os.path.join(OUTPUTS_DIR, 'thread_id_step2.txt'))
# thread_id = response_payload[]

payload = {
        "lesson": read_document(os.path.join(INPUTS_DIR, 'lesson.txt')),
        "goals_and_criteria": read_document(os.path.join(INPUTS_DIR, 'goals_and_criteria.txt')),
        "assessment_to_evaluate": read_document(os.path.join(INPUTS_DIR, 'assessment_to_eval.txt')),
        "strategic_document_input": strategic_doc,
        "studentid": "student1",
        "lessonid": "lesson0",
        "categories_to_evaluate": read_document(os.path.join(OUTPUTS_DIR, 'categories_to_evaluate.txt')),
        "user_context": "For a student that participates in the grade 6 classroom",
        "new_cks": new_cks if new_cks else "No CKS mapped at this point.",
        "thread_id": thread_id_step2
    }

#TUTOR SYS
# Invoke the Lambda function
try:
    response = lambda_client.invoke(
        FunctionName='generator-agent',  # Replace with your Lambda function name
        InvocationType='RequestResponse',  # Set to 'Event' for async
        Payload=json.dumps(payload)
    )

    # Process the response
    response_payload_step_3 = json.loads(response['Payload'].read())
    print("Response:", response_payload_step_3)
except Exception as err:
    print(f"Error invoking Lambda function: {err}")

File not found: .inputs/assessment_to_eval.txt
Response: {'statusCode': 200, 'headers': {'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*'}, 'body': "### Tailored Assessment for Agent Design and Operation\n\n#### Fill-in-the-Blank Exercises\n\n1. Modularity refers to the ability to break down tasks into smaller components, which allows agents to __________ more effectively.\n2. In AI, a good __________ is essential for making decisions that consider both short-term and long-term outcomes.\n3. __________ is crucial for structuring information about the world, while computational limits define the constraints within which agents must operate.\n4. Learning methodologies such as __________ are vital for agents to adapt and improve over time.\n5. Uncertainty in decision-making requires strategies to handle __________ environments.\n6. Preference analysis helps in prioritizing goals based on their __________ and likelihood.\n7. The number of agents in a system can signif

In [27]:
exercises_to_complete = response_payload_step_3['body']
print(exercises_to_complete)

### Tailored Assessment for Agent Design and Operation

#### Fill-in-the-Blank Exercises

1. Modularity refers to the ability to break down tasks into smaller components, which allows agents to __________ more effectively.
2. In AI, a good __________ is essential for making decisions that consider both short-term and long-term outcomes.
3. __________ is crucial for structuring information about the world, while computational limits define the constraints within which agents must operate.
4. Learning methodologies such as __________ are vital for agents to adapt and improve over time.
5. Uncertainty in decision-making requires strategies to handle __________ environments.
6. Preference analysis helps in prioritizing goals based on their __________ and likelihood.
7. The number of agents in a system can significantly affect dynamics and __________.
8. Interactivity enables __________ decision-making, which has profound implications for agent design.
9. Dimension interactions need to be s

#### Personalized Run


Once we have the first assessment to evaluate, we can run the following in order to get personalized feedback and map student knowledege gaps.

In [30]:
import boto3
import json
import uuid

#AWS credentials

thread_id = str(uuid.uuid4())

# Initialize boto3 Lambda client
lambda_client = boto3.client(
    'lambda',
    region_name='us-east-1',
    aws_access_key_id=AWS_ACCESS_KEY,  # Replace with your access key
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY  # Replace with your secret key
)

assessment_path = os.path.join(INPUTS_DIR, 'assessment_to_eval.txt')
assessment = read_document(assessment_path)

if assessment:
    cot_cat_extractor_input = {
        "lesson": read_document(os.path.join(INPUTS_DIR, 'lesson.txt')),
        "goals_and_criteria": read_document(os.path.join(INPUTS_DIR, 'goals_and_criteria.txt')),
        "assessment_to_evaluate": assessment,
        "studentid": "student1",
        "lessonid": "lesson0"
    }
    categories_path = os.path.join(OUTPUTS_DIR, 'categories_to_evaluate.txt')
    categories_to_evaluate = read_document(categories_path)
    evaluator_input = {
        **cot_cat_extractor_input,
        "categories_to_evaluate": categories_to_evaluate,
        "previous_interactions": "No previous interactions at the moment",
        "user_context": "For a student that participates in the grade 6 classroom"
    }

    try:
        response_step_1 = lambda_client.invoke(
            FunctionName='evaluator-agent',  # Replace with your Lambda function name
            InvocationType='RequestResponse',  # Set to 'Event' for async
            Payload=json.dumps(evaluator_input)
        )

        # Process the response
        response_payload_step_1 = json.loads(response_step_1['Payload'].read())
        print("Response:", response_payload_step_1)
    except Exception as err:
        print(f"Error invoking Lambda function: {err}")
    write_document(os.path.join(OUTPUTS_DIR, 'new_cks.txt'), new_cks)
    write_document(os.path.join(OUTPUTS_DIR, 'thread_id_step1.txt'), thread_id)

else:
    write_document(os.path.join(OUTPUTS_DIR, 'new_cks.txt'), '')
    write_document(os.path.join(OUTPUTS_DIR, 'thread_id_step1.txt'), '')
    click.echo("Step 1 skipped because assessment_to_eval.txt is empty.")

Response: {'statusCode': 200, 'headers': {'Content-Type': 'application/json', 'Access-Control-Allow-Origin': '*'}, 'body': '```json\n{\n    "CKS": {\n        "remember": {\n            "concept_one": {\n                "name": "Biodiversity",\n                "score": "D",\n                "reason": "The student has no knowledge of the topic.",\n                "state": "Needs Improvement"\n            },\n            "concept_two": {\n                "name": "Genetic Diversity",\n                "score": "D",\n                "reason": "The student has no knowledge of the topic.",\n                "state": "Needs Improvement"\n            },\n            "concept_three": {\n                "name": "Species Diversity",\n                "score": "D",\n                "reason": "The student has no knowledge of the topic.",\n                "state": "Needs Improvement"\n            },\n            "concept_four": {\n                "name": "Ecosystem Diversity",\n                "score": 

In [33]:
new_cks = response_payload_step_1['body']
print(response_payload_step_1['body'])

```json
{
    "CKS": {
        "remember": {
            "concept_one": {
                "name": "Biodiversity",
                "score": "D",
                "reason": "The student has no knowledge of the topic.",
                "state": "Needs Improvement"
            },
            "concept_two": {
                "name": "Genetic Diversity",
                "score": "D",
                "reason": "The student has no knowledge of the topic.",
                "state": "Needs Improvement"
            },
            "concept_three": {
                "name": "Species Diversity",
                "score": "D",
                "reason": "The student has no knowledge of the topic.",
                "state": "Needs Improvement"
            },
            "concept_four": {
                "name": "Ecosystem Diversity",
                "score": "D",
                "reason": "The student has no knowledge of the topic.",
                "state": "Needs Improvement"
            },
           

In [ ]:
# Define the payload to send to the Lambda function
payload_step_2 = {
        "lesson": read_document(os.path.join(INPUTS_DIR, 'lesson.txt')),
        "goals_and_criteria": read_document(os.path.join(INPUTS_DIR, 'goals_and_criteria.txt')),
        "user_context": "Grade 6 classroom",
        "studentid": "student1",
        "lessonid": "lesson0",
        "current_knowledge_state": new_cks if new_cks else "No student CKS is available",
        "old_cks": "Not yet provided",
        "categories_to_evaluate": categories_to_evaluate,
        "last_assessment": read_document(os.path.join(INPUTS_DIR, 'assessment_to_eval.txt')),
        "thread_id": thread_id_step1
    }

#TUTOR SYS
# Invoke the Lambda function
try:
    response_step_2 = lambda_client.invoke(
        FunctionName='tutorsys-s-analyst',  # Replace with your Lambda function name
        InvocationType='RequestResponse',  # Set to 'Event' for async
        Payload=json.dumps(payload_step_2)
    )

    # Process the response
    response_payload_step_2 = json.loads(response_step_2['Payload'].read())
    print("Response:", response_payload_step_2)
except Exception as err:
    print(f"Error invoking Lambda function: {err}")


In [ ]:
print(response_payload_step_2['body'])

In [ ]:
strategic_doc = response_payload_step_2['body']
thread_id_step2 = read_document(os.path.join(OUTPUTS_DIR, 'thread_id_step2.txt'))
# thread_id = response_payload[]

payload = {
        "lesson": read_document(os.path.join(INPUTS_DIR, 'lesson.txt')),
        "goals_and_criteria": read_document(os.path.join(INPUTS_DIR, 'goals_and_criteria.txt')),
        "assessment_to_evaluate": read_document(os.path.join(INPUTS_DIR, 'assessment_to_eval.txt')),
        "strategic_document_input": strategic_doc,
        "studentid": "student1",
        "lessonid": "lesson0",
        "categories_to_evaluate": read_document(os.path.join(OUTPUTS_DIR, 'categories_to_evaluate.txt')),
        "user_context": "For a student that participates in the grade 6 classroom",
        "new_cks": new_cks if new_cks else "No CKS mapped at this point.",
        "thread_id": thread_id_step2
    }

#TUTOR SYS
# Invoke the Lambda function
try:
    response = lambda_client.invoke(
        FunctionName='generator-agent',  # Replace with your Lambda function name
        InvocationType='RequestResponse',  # Set to 'Event' for async
        Payload=json.dumps(payload)
    )

    # Process the response
    response_payload_step_3 = json.loads(response['Payload'].read())
    print("Response:", response_payload_step_3)
except Exception as err:
    print(f"Error invoking Lambda function: {err}")